In [1]:
import numpy as np
from scipy import stats
import torch
import torch.optim as optim
import multiprocessing as mp
import logging
from datetime import datetime

from uniswapv3_simulator.pool import Uniswapv3Pool
from uniswapv3_simulator.optimization.environments import OneStepEnvironment, ScaleWrapper
from uniswapv3_simulator.optimization.ddpg.ddpg import (
    DDPG,
    DDPGTrainer,
    DeepActorModel,
    TrainArgs
)
from uniswapv3_simulator.optimization.ddpg.exploration_noise import GaussianProcess
from uniswapv3_simulator.optimization.ddpg.schedulers import ExponentialScheduler

timestamp = datetime.now().strftime('%y%m%d%H%M%S')
logging.basicConfig(
    level=logging.INFO,
    filename=f'./logs/rl_test_{timestamp}.log'
)
logging.getLogger('optimization').setLevel(logging.DEBUG)

In [2]:
SEED = 1234
seed_seq = np.random.SeedSequence(entropy=SEED)
seeds = seed_seq.generate_state(8)

init_price = 100
liquidity_bins = [70, 80, 90, 100, 110, 120, 130]

fees = stats.uniform(1e-4, 0.01 - 1e-4)
mu = stats.uniform(-0.05, 0.1)
sigma = stats.uniform(1e-4, 0.1 - 1e-4) 
alpha = stats.randint(1, 100 + 1)
beta = stats.randint(100, 1000 + 1)

fees.random_state = seeds[0]
mu.random_state = seeds[1]
sigma.random_state = seeds[2]
alpha.random_state = seeds[3]
beta.random_state = seeds[4]

n_sims_per_step = 500
n_jobs = mp.cpu_count() - 1

env = OneStepEnvironment(
    init_price, liquidity_bins,
    fees, mu, sigma, alpha, beta,
    n_sims_per_step=n_sims_per_step, 
    n_jobs=n_jobs, seed=seeds[5]
)

In [3]:
print('Random Variables')
print(f'fees:  mean={fees.mean():,.4f}, std={fees.std():,.4f}')
print(f'mu:    mean={mu.mean():,.4f}, std={mu.std():,.4f}')
print(f'sigma: mean={sigma.mean():,.4f}, std={sigma.std():,.4f}')
print(f'alpha: mean={alpha.mean():,.2f}, std={alpha.std():,.2f}')
print(f'beta:  mean={beta.mean():,.2f}, std={beta.std():,.2f}')

Random Variables
fees:  mean=0.0051, std=0.0029
mu:    mean=0.0000, std=0.0289
sigma: mean=0.0501, std=0.0288
alpha: mean=50.50, std=28.87
beta:  mean=550.00, std=260.10


In [4]:
def obs_scale_fn(obs):
    mu = np.array([0.025, 0.0, 0.0501, 50.50, 550.0])
    sigma = np.array([0.144, 0.0289, 0.0288, 28.87, 260.10])
    
    return (obs - mu) / sigma

def action_scale_fn(action):
    return action * 1e+5

def reward_scale_fn(reward):
    return reward

env = ScaleWrapper(env, obs_scale_fn, action_scale_fn, reward_scale_fn)

In [5]:
torch.manual_seed(seeds[6])
action_size = len(liquidity_bins) - 1
model = DeepActorModel(5, action_size, (128, 64), (128, 64))
agent = DDPG(
    model=model,
    gamma=0.99,
    tau=1e-3,
    optimizer=optim.Adam,
    actor_optimizer_kwargs={
        'lr': 1e-4,
        'weight_decay': 1e-5
    },
    critic_optimizer_kwargs={
        'lr': 1e-3,
        'weight_decay': 1e-5
    },
    clip_gradients=5.0
)
train_args = TrainArgs(
    train_steps=5000,
    batch_size=64, 
    memory_size=100000,
    exploration_noise=GaussianProcess,
    noise_kwargs={
        'size': (action_size, ), 
        'std': ExponentialScheduler(0.2, 0.01, 0.9995)
    },
    update_start=200,
    update_freq=4,
    clip_actions=(1e-6, np.inf),
    seed=seeds[7]
)
trainer = DDPGTrainer(agent, env, train_args)

In [6]:
model

DeepActorModel(
  (critic_layers): Sequential(
    (0): Linear(in_features=11, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=1, bias=True)
  )
  (actor_layers): Sequential(
    (0): Linear(in_features=5, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=6, bias=True)
  )
)

In [ ]:
%%time
rewards = trainer.train()

/Users/pspalding/Desktop/Phil/MFE/Courses/AFP/uniswap-v3-project/uniswapv3_simulator/optimization/traders.py:66: RuntimeWarning: invalid value encountered in sqrt
  return (-alpha + np.sqrt((alpha + beta * q0) ** 2 + 2 * beta * dy)) / beta
/Users/pspalding/Desktop/Phil/MFE/Courses/AFP/uniswap-v3-project/uniswapv3_simulator/optimization/traders.py:66: RuntimeWarning: invalid value encountered in sqrt
  return (-alpha + np.sqrt((alpha + beta * q0) ** 2 + 2 * beta * dy)) / beta
/Users/pspalding/Desktop/Phil/MFE/Courses/AFP/uniswap-v3-project/uniswapv3_simulator/optimization/traders.py:66: RuntimeWarning: invalid value encountered in sqrt
  return (-alpha + np.sqrt((alpha + beta * q0) ** 2 + 2 * beta * dy)) / beta
/Users/pspalding/Desktop/Phil/MFE/Courses/AFP/uniswap-v3-project/uniswapv3_simulator/optimization/traders.py:66: RuntimeWarning: invalid value encountered in sqrt
  return (-alpha + np.sqrt((alpha + beta * q0) ** 2 + 2 * beta * dy)) / beta
/Users/pspalding/Desktop/Phil/MFE/Course

Episode:    50 | Mean Score: -0.013091


/Users/pspalding/Desktop/Phil/MFE/Courses/AFP/uniswap-v3-project/uniswapv3_simulator/optimization/traders.py:66: RuntimeWarning: invalid value encountered in sqrt
  return (-alpha + np.sqrt((alpha + beta * q0) ** 2 + 2 * beta * dy)) / beta
/Users/pspalding/Desktop/Phil/MFE/Courses/AFP/uniswap-v3-project/uniswapv3_simulator/optimization/traders.py:66: RuntimeWarning: invalid value encountered in sqrt
  return (-alpha + np.sqrt((alpha + beta * q0) ** 2 + 2 * beta * dy)) / beta
/Users/pspalding/Desktop/Phil/MFE/Courses/AFP/uniswap-v3-project/uniswapv3_simulator/optimization/traders.py:66: RuntimeWarning: invalid value encountered in sqrt
  return (-alpha + np.sqrt((alpha + beta * q0) ** 2 + 2 * beta * dy)) / beta
/Users/pspalding/Desktop/Phil/MFE/Courses/AFP/uniswap-v3-project/uniswapv3_simulator/optimization/traders.py:66: RuntimeWarning: invalid value encountered in sqrt
  return (-alpha + np.sqrt((alpha + beta * q0) ** 2 + 2 * beta * dy)) / beta
/Users/pspalding/Desktop/Phil/MFE/Course

Episode:   100 | Mean Score: -0.012204
Episode:   150 | Mean Score: -0.012452


/Users/pspalding/Desktop/Phil/MFE/Courses/AFP/uniswap-v3-project/uniswapv3_simulator/optimization/traders.py:66: RuntimeWarning: invalid value encountered in sqrt
  return (-alpha + np.sqrt((alpha + beta * q0) ** 2 + 2 * beta * dy)) / beta
/Users/pspalding/Desktop/Phil/MFE/Courses/AFP/uniswap-v3-project/uniswapv3_simulator/optimization/traders.py:66: RuntimeWarning: invalid value encountered in sqrt
  return (-alpha + np.sqrt((alpha + beta * q0) ** 2 + 2 * beta * dy)) / beta


...

/Users/pspalding/Desktop/Phil/MFE/Courses/AFP/uniswap-v3-project/uniswapv3_simulator/optimization/traders.py:66: RuntimeWarning: invalid value encountered in sqrt
  return (-alpha + np.sqrt((alpha + beta * q0) ** 2 + 2 * beta * dy)) / beta
/Users/pspalding/Desktop/Phil/MFE/Courses/AFP/uniswap-v3-project/uniswapv3_simulator/optimization/traders.py:66: RuntimeWarning: invalid value encountered in sqrt
  return (-alpha + np.sqrt((alpha + beta * q0) ** 2 + 2 * beta * dy)) / beta


Episode:   900 | Mean Score: -0.011291


/Users/pspalding/Desktop/Phil/MFE/Courses/AFP/uniswap-v3-project/uniswapv3_simulator/optimization/traders.py:66: RuntimeWarning: invalid value encountered in sqrt
  return (-alpha + np.sqrt((alpha + beta * q0) ** 2 + 2 * beta * dy)) / beta
/Users/pspalding/Desktop/Phil/MFE/Courses/AFP/uniswap-v3-project/uniswapv3_simulator/optimization/traders.py:66: RuntimeWarning: invalid value encountered in sqrt
  return (-alpha + np.sqrt((alpha + beta * q0) ** 2 + 2 * beta * dy)) / beta
/Users/pspalding/Desktop/Phil/MFE/Courses/AFP/uniswap-v3-project/uniswapv3_simulator/optimization/traders.py:66: RuntimeWarning: invalid value encountered in sqrt
  return (-alpha + np.sqrt((alpha + beta * q0) ** 2 + 2 * beta * dy)) / beta
/Users/pspalding/Desktop/Phil/MFE/Courses/AFP/uniswap-v3-project/uniswapv3_simulator/optimization/traders.py:66: RuntimeWarning: invalid value encountered in sqrt
  return (-alpha + np.sqrt((alpha + beta * q0) ** 2 + 2 * beta * dy)) / beta
/Users/pspalding/Desktop/Phil/MFE/Course

Episode:   950 | Mean Score: -0.012548


/Users/pspalding/Desktop/Phil/MFE/Courses/AFP/uniswap-v3-project/uniswapv3_simulator/optimization/traders.py:66: RuntimeWarning: invalid value encountered in sqrt
  return (-alpha + np.sqrt((alpha + beta * q0) ** 2 + 2 * beta * dy)) / beta
/Users/pspalding/Desktop/Phil/MFE/Courses/AFP/uniswap-v3-project/uniswapv3_simulator/optimization/traders.py:66: RuntimeWarning: invalid value encountered in sqrt
  return (-alpha + np.sqrt((alpha + beta * q0) ** 2 + 2 * beta * dy)) / beta
/Users/pspalding/Desktop/Phil/MFE/Courses/AFP/uniswap-v3-project/uniswapv3_simulator/optimization/traders.py:66: RuntimeWarning: invalid value encountered in sqrt
  return (-alpha + np.sqrt((alpha + beta * q0) ** 2 + 2 * beta * dy)) / beta
/Users/pspalding/Desktop/Phil/MFE/Courses/AFP/uniswap-v3-project/uniswapv3_simulator/optimization/traders.py:66: RuntimeWarning: invalid value encountered in sqrt
  return (-alpha + np.sqrt((alpha + beta * q0) ** 2 + 2 * beta * dy)) / beta
/Users/pspalding/Desktop/Phil/MFE/Course